## AutoGen Studio Agent Workflow API Example

This notebook focuses on demonstrating capabilities of the autogen studio workflow python api.  

- Declarative Specification of an Agent Team
- Loading the specification and running the resulting agent

 

In [1]:
from autogenstudio.teammanager import TeamManager 
 
wm = TeamManager()  
result = await wm.run(task="What is the weather in New York?", team_config="team.json") 
print(result)

task_result=TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?'), ToolCallMessage(source='writing_agent', models_usage=RequestUsage(prompt_tokens=65, completion_tokens=15), content=[FunctionCall(id='call_1LrIS8t1wuK3KzDPP4DuwuiQ', arguments='{"city":"New York"}', name='get_weather')]), ToolCallResultMessage(source='writing_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_1LrIS8t1wuK3KzDPP4DuwuiQ')]), TextMessage(source='writing_agent', models_usage=RequestUsage(prompt_tokens=97, completion_tokens=14), content='The current weather in New York is 73 degrees and sunny.'), TextMessage(source='writing_agent', models_usage=RequestUsage(prompt_tokens=123, completion_tokens=9), content='What is the weather in Los Angeles?')], stop_reason='Maximum number of messages 5 reached, current message count: 5') usage='' duration=2.2458291053771973


In [2]:
result_stream =  wm.run_stream(task="What is the weather in New York?", team_config="team.json") 
async for response in result_stream:
    print(response)

source='user' models_usage=None content='What is the weather in New York?'
source='writing_agent' models_usage=RequestUsage(prompt_tokens=65, completion_tokens=15) content=[FunctionCall(id='call_Otg8Exxp7uAcsZnRyTWRYGkb', arguments='{"city":"New York"}', name='get_weather')]
source='writing_agent' models_usage=None content=[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', call_id='call_Otg8Exxp7uAcsZnRyTWRYGkb')]
source='writing_agent' models_usage=RequestUsage(prompt_tokens=97, completion_tokens=14) content='The weather in New York is currently 73 degrees and sunny.'
source='writing_agent' models_usage=RequestUsage(prompt_tokens=123, completion_tokens=16) content='Would you like to know anything else about the weather or any other information?'
task_result=TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?'), ToolCallMessage(source='writing_agent', models_usage=RequestUsage(prompt_tokens=65, c

## AutoGen Studio Database API

Api for creating objects and serializing to a database.

In [3]:
from autogenstudio.database import DatabaseManager 

# create a database
dbmanager = DatabaseManager(engine_uri="sqlite:///test.db")
dbmanager.create_db_and_tables() 

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2024-11-09 11:55:02.090 | INFO     | autogenstudio.database.schema_manager:upgrade_schema:390 - Schema upgraded successfully to head
2024-11-09 11:55:02.090 | INFO     | autogenstudio.database.db_manager:__init__:34 - Database schema was upgraded automatically
2024-11-09 11:55:02.092 | INFO     | autogenstudio.database.db_manager:create_db_and_tables:108 - Database tables created successfully


In [4]:

from sqlmodel import Session, text, select
from autogenstudio.datamodel import Model, ModelConfig, ModelTypes, Team, TeamConfig, TeamTypes, Agent, AgentConfig, AgentTypes, Tool, ToolConfig, LinkTypes,ToolTypes

user_id = "guestuser@gmail.com"
from autogenstudio.datamodel import ModelConfig, Model, TeamConfig, Team, Tool, Agent, AgentConfig, TerminationConfig, TerminationTypes, ModelTypes, TeamTypes, AgentTypes, ToolConfig, LinkTypes, TerminationTypes

gpt4_model = Model(user_id=user_id, config= ModelConfig(model="gpt-4o-2024-08-06", model_type=ModelTypes.OPENAI).model_dump() )

weather_tool = Tool(user_id=user_id, config=ToolConfig(name="get_weather", description="Get the weather for a city", content="async def get_weather(city: str) -> str:\n    return f\"The weather in {city} is 73 degrees and Sunny.\"",tool_type=ToolTypes.PYTHON_FUNCTION).model_dump() )

adding_tool = Tool(user_id=user_id, config=ToolConfig(name="add", description="Add two numbers", content="async def add(a: int, b: int) -> int:\n    return a + b", tool_type=ToolTypes.PYTHON_FUNCTION).model_dump() )

writing_agent = Agent(user_id=user_id, 
                      config=AgentConfig(
                          name="writing_agent", 
                          tools=[weather_tool.config], 
                          agent_type=AgentTypes.ASSISTANT,
                          model_client=gpt4_model.config
                          ).model_dump()
                    )

team = Team(user_id=user_id, config=TeamConfig(
    name="weather_team",
    participants=[writing_agent.config],
    termination_condition=TerminationConfig(termination_type=TerminationTypes.MAX_MESSAGES, max_messages=5).model_dump(),
    team_type=TeamTypes.ROUND_ROBIN
    ).model_dump()
)

with Session(dbmanager.engine) as session:
    session.add(gpt4_model)
    session.add(weather_tool)
    session.add(adding_tool)
    session.add(writing_agent)
    session.add(team)
    session.commit()

    dbmanager.link(LinkTypes.AGENT_MODEL, writing_agent.id, gpt4_model.id)
    dbmanager.link(LinkTypes.AGENT_TOOL, writing_agent.id, weather_tool.id)
    dbmanager.link(LinkTypes.AGENT_TOOL, writing_agent.id, adding_tool.id)
    dbmanager.link(LinkTypes.TEAM_AGENT, team.id, writing_agent.id)


In [5]:
all_teams = dbmanager.get(Team)
print(len(all_teams.data), "teams in database")

43 teams in database


## Configuration Manager

Helper class to mostly import teams/agents/models/tools etc into a database.

In [6]:
from autogenstudio.database import ConfigurationManager  
config_manager = ConfigurationManager(dbmanager)
 

In [7]:
result = await config_manager.import_component("team.json", user_id="user_id", check_exists=True)
print(result)

message="team with team_type='TeamTypes.ROUND_ROBIN' and name='weather_team' already exists" status=True data={'id': 42}


In [8]:
result = await config_manager.import_directory(".", user_id="user_id", check_exists=False)
print(result)

message='Directory import complete' status=True data=[{'component': 'team', 'status': True, 'message': 'Team Created Successfully', 'id': 44}]


In [9]:
all_teams = dbmanager.get(Team)
print(len(all_teams.data), "teams in database")

44 teams in database
